In [1]:
import os
%pwd
os.chdir("../")
os.getcwd()

'/Users/varundesai/Visual_Studio_Varun/Projects/Chicken-Disease-Classification'

In [2]:
#Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from chickenDiseaseClassification.constants import *
from chickenDiseaseClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_augmentation= params.AUGMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return training_config

In [13]:
# Component
import time

In [14]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_weights_only=True,
            save_best_only=True,
            monitor="val_accuracy",
            mode="max",
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [15]:
import os
import urllib.request as request
from zipfile import ZipInfo
import tensorflow as tf

In [16]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
            )
        else:
            train_datagenerator = valid_datagenerator
        
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    
    def train(self,callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.valid_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.valid_steps,
            callbacks = callback_list
        )
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [17]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
except Exception as e:
    raise e

[2024-02-27 14:20:37,475: INFO: common: 31: yaml file: config/config.yaml is loaded successfully]
[2024-02-27 14:20:37,477: INFO: common: 31: yaml file: params.yaml is loaded successfully]
[2024-02-27 14:20:37,478: INFO: common: 58: Creating directory: artifacts]
[2024-02-27 14:20:37,478: INFO: common: 58: Creating directory: artifacts/prepare_callbacks/checkpoint_dir]
[2024-02-27 14:20:37,479: INFO: common: 58: Creating directory: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-02-27 14:20:37,480: INFO: common: 58: Creating directory: artifacts/training]
Found 78 images belonging to 2 classes.
Found 390 images belonging to 2 classes.
Epoch 1/50
24/24 [==============================] - 40s 2s/step - loss: 14.8556 - accuracy: 0.7647 - val_loss: 2.7775 - val_accuracy: 0.4219
Epoch 2/50
24/24 [==============================] - 43s 2s/step - loss: 4.1220 - accuracy: 0.8984 - val_loss: 2.6802 - val_accuracy: 0.4062
Epoch 3/50
24/24 [==============================] - 45s 2s/step - los

/Users/varundesai/anaconda3/envs/cdcEnv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
